First install pgvector in pgsql
https://github.com/pgvector/pgvector?tab=readme-ov-file#installation


CREATE EXTENSION IF NOT EXISTS vector;
ALTER TABLE localnews.articles ADD COLUMN vector_2d vector(2);
ALTER TABLE localnews.articles ADD COLUMN vector vector(768);
ALTER TABLE localnews.articles ADD COLUMN similar_matches jsonb;


In [16]:
import psycopg2
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname="decodeMT",
    user="postgres",
    password="rufy100",
)
cursor = conn.cursor()

# Fetch articles
cursor.execute('select entryid, article from localnews.articles order by parseddate')
rows = cursor.fetchall()

# Load the sentence transformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Process each article and update its vector in the database
for row in tqdm(rows):
    entryid, article = row  # Access the correct columns
    encoding = model.encode(article)  # Get the article text

    # Convert numpy array to list and format it as a PostgreSQL array
    encoding_list = encoding.tolist()

    # Update the article's vector column
    cursor.execute('update localnews.articles set vector = %s where entryid = %s', (encoding_list, entryid))

    # Commit changes to the database
    conn.commit()

# Close the connection
cursor.close()
conn.close()


100%|██████████| 503235/503235 [4:53:32<00:00, 28.57it/s]   


In [3]:
import psycopg2
from tqdm import tqdm
import numpy as np
from sklearn.decomposition import PCA
import ast
import umap


# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname="decodeMT",
    user="postgres",
    password="rufy100",
)
cursor = conn.cursor()

# Fetch articles
cursor.execute('select entryid, vector from localnews.articles;')
rows = cursor.fetchall()

vectors = [ast.literal_eval(row[1]) for row in rows]
vectors = np.array(vectors, dtype=float)

# pca = PCA(n_components=2)
# reduced_vectors = pca.fit_transform(vectors)
umap_reducer = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1, random_state=42)
reduced_vectors = umap_reducer.fit_transform(vectors)

# Process each article and update its vector in the database
for row, vector in tqdm(zip(rows, reduced_vectors)):

    cursor.execute('update localnews.articles set vector_2d = %s where entryid = %s', (vector.tolist(), row[0]))

    conn.commit()

# Close the connection
cursor.close()
conn.close()


c:\Users\alang\miniconda3\envs\news_two\Lib\site-packages\umap\umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
503235it [02:36, 3207.23it/s]


In [7]:
import psycopg2
from tqdm import tqdm
import numpy as np
from sklearn.decomposition import PCA
import ast
import json


# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname="decodeMT",
    user="postgres",
    password="rufy100",
)

conn_sim = psycopg2.connect(
    dbname="decodeMT",
    user="postgres",
    password="rufy100",
)
cursor = conn.cursor()

conn_sim.autocommit = True
cursor_sim = conn_sim.cursor()

# Fetch articles
cursor.execute("select entryid from localnews.articles order by parseddate , entryid");
rows = cursor.fetchall()
for row in tqdm(rows):
    cursor_sim.execute("""
                        WITH date_range AS (
                            SELECT parseddate
                            FROM localnews.articles
                            WHERE entryid = %s
                        ),
                        temp_table AS (
                            SELECT entryid, vector_2d, vector <=> (SELECT vector FROM localnews.articles WHERE entryid = %s) AS similarity
                            FROM localnews.articles a, date_range
                            WHERE vector IS NOT NULL
                            AND a.parseddate BETWEEN date_range.parseddate - INTERVAL '6 months'
                            AND date_range.parseddate + INTERVAL '6 months'
                        )
                        SELECT entryid, vector_2d, (1-similarity) AS "similar"
                        FROM temp_table
                        WHERE (1-similarity) >= 0.7
                        AND entryid != %s
                        ORDER BY "similar" DESC;
                        """, (row[0], row[0], row[0]))

    result = cursor_sim.fetchall()

    processed_result = [
        {
            "to": row[0],
            "x": round(ast.literal_eval(row[1])[0], 2),
            "y": round(ast.literal_eval(row[1])[1], 2),
            "val": round(row[2],2)
        }
        for row in result
    ]

    cursor_sim.execute('update localnews.articles set similar_articles = %s where entryid = %s', (json.dumps(processed_result), row[0]))

100%|██████████| 503233/503233 [39:44:44<00:00,  3.52it/s]    


In [3]:
cursor.close()

Then create a table copy only required content. The final query is how to retrieve data.

CREATE TABLE IF NOT EXISTS news
(
    entryid bigint NOT NULL,
    source character varying(6),
    title character varying(2048),
    parseddate date,
    link text,
    vector_2d text,
    similar_articles text,
    CONSTRAINT art_pk PRIMARY KEY (entryid)
)


insert into news
select entryId, source, title, cast(parseddate as date), link, cast(vector_2d as text)
		, cast(similar_articles as text)
from localnews.articles


WITH relevant_ids AS (
    SELECT entryid 
    from news
   WHERE parseddate BETWEEN '2020-01-14'::DATE - INTERVAL '2 days' 
                      AND '2020-01-14'::DATE + INTERVAL '2 days'
),
filtered_matches AS (
    SELECT 
        a.entryid, 
        a.source, 
        a.title, 
        TO_CHAR(a.parseddate::timestamp, 'yyyy-mm-dd') AS date, 
        a.link,
        split_part(trim(both '[]' from a.vector_2d), ',', 1)::float AS x,
        split_part(trim(both '[]' from a.vector_2d), ',', 2)::float AS y,
        jsonb_agg(
            jsonb_build_object('to', elements.value ->> 'to', 'val', elements.value ->> 'val')
        ) AS filtered_similar_matches
    FROM 
        news a
    JOIN 
        jsonb_array_elements(a.similar_articles::jsonb) AS elements ON (elements.value ->> 'to')::int IN (SELECT entryid FROM relevant_ids)
    JOIN 
        relevant_ids r ON a.entryid = r.entryid
    WHERE 
        (elements.value ->> 'to')::int <> a.entryid
	
    GROUP BY 
        a.entryid, a.source, a.title, TO_CHAR(a.parseddate::timestamp, 'yyyy-mm-dd'), a.link, a.vector_2d
)
SELECT * FROM filtered_matches order by entryid;
